In [0]:
#https://www.leafly.com/strains/lists/condition/pain

!pip install requests-html > /dev/null

In [0]:
from pygments import highlight
from pygments.lexers import HtmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
from urllib.request import urlopen
from requests.adapters import HTTPAdapter
import requests
import re
import json



In [0]:
def pretty_print_html(html_string):
  html_string = BeautifulSoup(html_string, 'html.parser').prettify()
  pygments_css = pygments_css=HtmlFormatter().get_style_defs('.highlight')
  data = highlight(html_string, HtmlLexer(), HtmlFormatter())
  return display(HTML(f'<style>{pygments_css}</style>{data}'))

In [0]:
!wget 'https://www.leafly.com/strains/lists/condition/pain'

--2020-03-05 20:02:56--  https://www.leafly.com/strains/lists/condition/pain
Resolving www.leafly.com (www.leafly.com)... 104.18.203.76, 104.18.202.76, 2606:4700::6812:cb4c, ...
Connecting to www.leafly.com (www.leafly.com)|104.18.203.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘pain.1’

pain.1                  [ <=>                ] 184.35K  --.-KB/s    in 0.07s   

2020-03-05 20:02:57 (2.74 MB/s) - ‘pain.1’ saved [188775]



In [0]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.leafly.com/strains/lists/condition/pain')

In [0]:
r

<Response [200]>

In [0]:
r.html.html

'<!DOCTYPE html><html lang="en"><head><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-standard-thin.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-standard-medium.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://public.leafly.com/fonts/gt-america/gt-america-condensed-medium.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="dns-prefetch" href="https://public.leafly.com"/><link rel="dns-prefetch" href="https://public.leafly.com"/><script type="text/javascript" src="https://public.leafly.com/scripts/split-10.10.0.min.js" defer=""></script><script type="text/javascript" src="https://public.leafly.com/cephalopod/dist/vendors.87e2f9ad0784ea148b7c.js" defer=""></script><script type="text/javascript" src="https://public.leafly.com/cephalopod/dist/helpers.c20c4673027656787dc7.js" defer=""></script><script t

In [0]:
pretty_print_html(r.html.html)

In [0]:
#naming our html document
document = r.html

In [0]:
#finding each strain name
strain_name = document.find('.strain-tile')
strain_name

[<Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/hurkle'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/brand-x'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/lake-of-fire'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/snow-monster'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/stevie-wonder'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/tora-bora'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/digweed'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/mazar-kush'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/purple-arrow'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative') href='/strains/avi-dekel'>,
 <Element 'a' class=('strain-tile', 'justify-start', 'relative'

In [0]:
#selecting the first strain name 
first_strain_name = strain_name[0]

In [0]:
#pretty print of the strain name
pretty_print_html(first_strain_name.html)

In [0]:
#pinpointed the text of the strain name!
first_strain_name.find('.strain-tile__name', first = True).text

'Hurkle'

In [0]:
#figured out how to get the text only
first_strain_name.find('span', first = True).text

'Relaxed'

In [0]:
#pulling the attribute of the link only
first_strain_name.attrs['href']

'/strains/hurkle'

In [0]:
#getting ready to create the dataframe
strain_list = []
for strain in strain_name:
  name = strain.find('.strain-tile__name')[0].text
  affect = strain.find('span')[0].text
  link = strain.attrs['href']
  strain_list.append({'Name':name, 'Affect':affect, 'Links': link})

In [0]:
df = pd.DataFrame(strain_list)
df

,Name,Affect,Links
0,Hurkle,Relaxed,/strains/hurkle
1,Brand X,Hungry,/strains/brand-x
2,Lake of Fire,Aroused,/strains/lake-of-fire
3,Snow Monster,Giggly,/strains/snow-monster
4,Stevie Wonder,Focused,/strains/stevie-wonder
5,Tora Bora,Relaxed,/strains/tora-bora
6,Digweed,Giggly,/strains/digweed
7,Mazar Kush,Sleepy,/strains/mazar-kush
8,Purple Arrow,Uplifted,/strains/purple-arrow
9,Avi-Dekel,Focused,/strains/avi-dekel


In [0]:
df['Links'] = df['Links'].apply(lambda x: '<a href="http://www.leafly.com/{0}">Learn More'.format(x))

HTML(df.to_html(escape=False))

,Name,Affect,Links
0,Hurkle,Relaxed,Learn More
1,Brand X,Hungry,Learn More
2,Lake of Fire,Aroused,Learn More
3,Snow Monster,Giggly,Learn More
4,Stevie Wonder,Focused,Learn More
5,Tora Bora,Relaxed,Learn More
6,Digweed,Giggly,Learn More
7,Mazar Kush,Sleepy,Learn More
8,Purple Arrow,Uplifted,Learn More
9,Avi-Dekel,Focused,Learn More


In [0]:
df

,Name,Affect,Links
0,Hurkle,Relaxed,"<a href=""http://www.leafly.com//strains/hurkle..."
1,Brand X,Hungry,"<a href=""http://www.leafly.com//strains/brand-..."
2,Lake of Fire,Aroused,"<a href=""http://www.leafly.com//strains/lake-o..."
3,Snow Monster,Giggly,"<a href=""http://www.leafly.com//strains/snow-m..."
4,Stevie Wonder,Focused,"<a href=""http://www.leafly.com//strains/stevie..."
5,Tora Bora,Relaxed,"<a href=""http://www.leafly.com//strains/tora-b..."
6,Digweed,Giggly,"<a href=""http://www.leafly.com//strains/digwee..."
7,Mazar Kush,Sleepy,"<a href=""http://www.leafly.com//strains/mazar-..."
8,Purple Arrow,Uplifted,"<a href=""http://www.leafly.com//strains/purple..."
9,Avi-Dekel,Focused,"<a href=""http://www.leafly.com//strains/avi-de..."


In [0]:
df.to_csv('pain_strain_list')